# Tutorial: converting existing code to serverless without internal code modifications

In [1]:
import os
import warnings

warnings.filterwarnings('ignore')

from quantum_serverless import QuantumServerless, run_qiskit_remote, get, put

---

There are couple of ways to make your code running as serverless code:
- wrapping entire functions / classes as actors or functions
- monkey-patching existing class functions to swap some parts of code

### Approach #1: wrapping function / classes as tasks / actors

We will work with qiskit and try to wrap some functions / classes as tasks / actors

In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit.providers import Backend
from qiskit.circuit.random import random_circuit
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeVigo, FakeAlmaden, FakeBrooklyn, FakeCasablanca

circuit = random_circuit(5, 3)
backend = AerSimulator.from_backend(FakeAlmaden())

circuit.draw()

┌───┐ ┌───┐                                
q_0: ─┤ X ├─┤ X ├──■─────────────────────────────
     ┌┴───┴┐└─┬─┘  │  ┌─────────────────────────┐
q_1: ┤ Sdg ├──■────┼──┤ U3(1.417,2.9487,2.2238) ├
     └─────┘  │    │  └─────────────────────────┘
q_2: ───■─────■────■─────────────────────────────
        │        ┌─┴─┐                           
q_3: ───■─────■──┤ X ├───────────────────────────
      ┌─┴─┐ ┌─┴─┐├───┤                           
q_4: ─┤ X ├─┤ Y ├┤ T ├───────────────────────────
      └───┘ └───┘└───┘

We have transpile function. Let's start with transpilation function.

You can call `transpile(circuits, backend)` to compile circuits to specific backend locally. 

In [3]:
# let's transpile function to see what it does
transpiled_circuit = transpile(circuit, backend)

transpiled_circuit.draw(idle_wires=False, fold=-1)

global phase: 0
           ┌──────────┐                                                                                                                                            ┌─────────┐                                         ┌───┐┌──────────┐     ┌───┐┌─────────────────────────┐                                                                                                                       
 q_1 -> 9 ─┤ U1(-π/2) ├───────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──┤ U1(π/4) ├─────────────────────────────────────────┤ X ├┤ U1(-π/4) ├─────┤ X ├┤ U3(1.417,2.9487,2.2238) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
           └──────────┘       │                                   ┌─────────┐                                       ┌───┐    ┌──────────┐┌───┐                  │  └─────────┘                                         └─┬─┘└──┬───┬───┘     └─┬─┘└─────────────────────────┘                                                  ┌─────────┐                     ┌───┐     ┌───┐                      
q_3 -> 12 ───────────────■────┼────────────────────────────────■──┤ U1(π/4) ├───────────────────────────────────────┤ X ├────┤ U1(-π/4) ├┤ X ├──────────────────┼───────────────────────────────────────────────────■────┼─────┤ X ├──────■────┼───────────────────────────────────────────■────────────────────────────────■──┤ U1(π/4) ├─────────────────────┤ X ├──■──┤ X ├──────────────────────
           ┌─────────┐ ┌─┴─┐  │  ┌──────────┐┌───┐┌─────────┐┌─┴─┐├─────────┴┐┌───┐┌─────────────┐┌───┐     ┌───┐   └─┬─┘    ├─────────┬┘└─┬─┘                  │                                       ┌───┐     ┌─┴─┐  │     └─┬─┘    ┌─┴─┐  │                             ┌─────────┐ ┌─┴─┐┌──────────┐┌───┐┌─────────┐┌─┴─┐├─────────┴┐┌───┐┌─────────────┐└─┬─┘┌─┴─┐└─┬─┘┌───┐┌──────────┐┌───┐
q_4 -> 13 ─┤ U2(0,π) ├─┤ X ├──┼──┤ U1(-π/4) ├┤ X ├┤ U1(π/4) ├┤ X ├┤ U1(-π/4) ├┤ X ├┤ U2(0,-3π/4) ├┤ X ├──■──┤ X ├─────■──────┤ U1(π/4) ├───■────■───────────────┼────────────────■───────────────────■──┤ X ├──■──┤ X ├──┼───────■──────┤ X ├──┼───────────────■─────────────┤ U2(0,π) ├─┤ X ├┤ U1(-π/4) ├┤ X ├┤ U1(π/4) ├┤ X ├┤ U1(-π/4) ├┤ X ├┤ U2(0,-3π/4) ├──■──┤ X ├──■──┤ X ├┤ U1(-π/4) ├┤ X ├
          ┌┴─────────┴┐└───┘┌─┴─┐├──────────┤└─┬─┘└─────────┘└───┘└──────────┘└─┬─┘└─────────────┘└─┬─┘  │  └─┬─┘            └─────────┘      ┌─┴─┐┌─────────┐┌─┴─┐┌──────────┐┌─┴─┐┌─────────────┐┌─┴─┐└─┬─┘┌─┴─┐└───┘  │  ┌─────────┐ └───┘  │               │             └─────────┘ └───┘└──────────┘└─┬─┘└─────────┘└───┘└──────────┘└─┬─┘└─────────────┘     └───┘     └─┬─┘├─────────┬┘└─┬─┘
q_0 -> 14 ┤ U2(-π,-π) ├─────┤ X ├┤ U1(-π/4) ├──┼────────────────────────────────┼───────────────────┼────┼────┼───────────────────────────────┤ X ├┤ U1(π/4) ├┤ X ├┤ U1(-π/4) ├┤ X ├┤ U2(0,-3π/4) ├┤ X ├──■──┤ X ├───────■──┤ U1(π/4) ├────────■───────────────┼────────────────────────────────────────────■────────────────────────────────■──────────────────────────────────■──┤ U1(π/4) ├───■──
          └───────────┘     └───┘└──────────┘  │                                │                   │  ┌─┴─┐  │  ┌──────────┐                 └───┘└─────────┘└───┘└──────────┘└───┘└─────────────┘└───┘     └───┘          └─────────┘                      ┌─┴─┐           ┌──────────┐                                                                                          └─────────┘      
q_2 -> 18 ─────────────────────────────────────■────────────────────────────────■───────────────────■──┤ X ├──■──┤ U1(-π/2) ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├───────────┤ U1(3π/4) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                              

We can turn this function in ray remote function, that can be executed in parallel on configured machine / cluster

In [4]:
remote_transpile = run_qiskit_remote()(transpile)

Now we have remote transpile function, we can try it out. But before let's create serverless class which will gives us executon context.

In [5]:
serverless = QuantumServerless()
serverless

<QuantumServerless | providers [local]>

In [6]:
with serverless:
    # execute remote transpile function and get back pointer to remote task, 
    #    so we can fetch results out of it
    task = remote_transpile(circuit, backend)
    print(f"Pointer to task: {task}")
    
    # get actual results from task, 
    #    which will be our transpiled circuit
    remotely_transpiled_circuits = get(task)

remotely_transpiled_circuits.draw(idle_wires=False, fold=-1)

Pointer to task: ObjectRef(c8ef45ccd0112571ffffffffffffffffffffffff0100000001000000)


global phase: 0
           ┌──────────┐                                                                                                                                            ┌─────────┐                                         ┌───┐┌──────────┐     ┌───┐┌─────────────────────────┐                                                                                                                       
 q_1 -> 9 ─┤ U1(-π/2) ├───────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──┤ U1(π/4) ├─────────────────────────────────────────┤ X ├┤ U1(-π/4) ├─────┤ X ├┤ U3(1.417,2.9487,2.2238) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
           └──────────┘       │                                   ┌─────────┐                                       ┌───┐    ┌──────────┐┌───┐                  │  └─────────┘                                    ┌───┐└─┬─┘└──────────┘┌───┐└─┬─┘└─────────────────────────┘                                                  ┌─────────┐                          ┌───┐                           
q_3 -> 12 ───────────────■────┼────────────────────────────────■──┤ U1(π/4) ├───────────────────────────────────────┤ X ├────┤ U1(-π/4) ├┤ X ├──────────────────┼─────────────────────────────────────────────────┤ X ├──┼───────■──────┤ X ├──┼───────────────────────────────────────────■────────────────────────────────■──┤ U1(π/4) ├───────────────────────■──┤ X ├──■────────────────────────
           ┌─────────┐ ┌─┴─┐  │  ┌──────────┐┌───┐┌─────────┐┌─┴─┐├─────────┴┐┌───┐┌─────────────┐┌───┐     ┌───┐   └─┬─┘    ├─────────┬┘└─┬─┘                  │                                       ┌───┐     └─┬─┘  │     ┌─┴─┐    └─┬─┘  │                             ┌─────────┐ ┌─┴─┐┌──────────┐┌───┐┌─────────┐┌─┴─┐├─────────┴┐┌───┐┌─────────────┐┌─┴─┐└─┬─┘┌─┴─┐┌───┐┌──────────┐┌───┐
q_4 -> 13 ─┤ U2(0,π) ├─┤ X ├──┼──┤ U1(-π/4) ├┤ X ├┤ U1(π/4) ├┤ X ├┤ U1(-π/4) ├┤ X ├┤ U2(0,-3π/4) ├┤ X ├──■──┤ X ├─────■──────┤ U1(π/4) ├───■────■───────────────┼────────────────■───────────────────■──┤ X ├──■────■────┼─────┤ X ├──────■────┼───────────────■─────────────┤ U2(0,π) ├─┤ X ├┤ U1(-π/4) ├┤ X ├┤ U1(π/4) ├┤ X ├┤ U1(-π/4) ├┤ X ├┤ U2(0,-3π/4) ├┤ X ├──■──┤ X ├┤ X ├┤ U1(-π/4) ├┤ X ├
          ┌┴─────────┴┐└───┘┌─┴─┐├──────────┤└─┬─┘└─────────┘└───┘└──────────┘└─┬─┘└─────────────┘└─┬─┘  │  └─┬─┘            └─────────┘      ┌─┴─┐┌─────────┐┌─┴─┐┌──────────┐┌─┴─┐┌─────────────┐┌─┴─┐└─┬─┘┌─┴─┐       │  ┌──┴───┴──┐        │               │             └─────────┘ └───┘└──────────┘└─┬─┘└─────────┘└───┘└──────────┘└─┬─┘└─────────────┘└───┘     └───┘└─┬─┘├─────────┬┘└─┬─┘
q_0 -> 14 ┤ U2(-π,-π) ├─────┤ X ├┤ U1(-π/4) ├──┼────────────────────────────────┼───────────────────┼────┼────┼───────────────────────────────┤ X ├┤ U1(π/4) ├┤ X ├┤ U1(-π/4) ├┤ X ├┤ U2(0,-3π/4) ├┤ X ├──■──┤ X ├───────■──┤ U1(π/4) ├────────■───────────────┼────────────────────────────────────────────■────────────────────────────────■──────────────────────────────────■──┤ U1(π/4) ├───■──
          └───────────┘     └───┘└──────────┘  │                                │                   │  ┌─┴─┐  │  ┌──────────┐                 └───┘└─────────┘└───┘└──────────┘└───┘└─────────────┘└───┘     └───┘          └─────────┘                      ┌─┴─┐           ┌──────────┐                                                                                          └─────────┘      
q_2 -> 18 ─────────────────────────────────────■────────────────────────────────■───────────────────■──┤ X ├──■──┤ U1(-π/2) ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├───────────┤ U1(3π/4) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                              

Because we have this function as ray remote function, we can run multiple of them in parallel

In [7]:
with serverless:
    # let's run 5 circuits transpilations in parallel
    tasks = [
        remote_transpile(random_circuit(5, i + 1), backend)
        for i in range(5)
    ]

    # get all results when ready
    transpiled_circuits = get(tasks)
    
# look at our final transpiled circuit
transpiled_circuits[-1].draw(idle_wires=False, fold=-1)

global phase: 3.1306
                               ┌───┐                 ┌───┐┌─────────────────────┐                                                                                                                                                                                                            ┌─────────┐                                                                                                   ┌───┐                    ┌───┐                                                                                     
 q_3 -> 9 ─────────────────────┤ X ├─────────■───────┤ X ├┤ U3(1.1556,π/2,-π/2) ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────────────────────────■──────────────────────────────────■──┤ U1(π/4) ├───■───────────────────────────────────────────────────────────────────────────────────────────────┤ X ├────────────■───────┤ X ├─────────────────────────────────────────────────────────────────────────────────────
                               └─┬─┘         │       └─┬─┘└─────────────────────┘     ┌─────────┐                                    ┌───┐┌──────────┐┌───┐┌───────────────────────────┐                              │                                │                                  │  └─────────┘   │                                                                                               └─┬─┘            │       └─┬─┘                                                                                     
q_0 -> 12 ───────────────────────┼───────────┼─────────┼───────────────────────────■──┤ U1(π/4) ├────────────────────────────────────┤ X ├┤ U1(-π/4) ├┤ X ├┤ U3(2.2178,2.7588,0.70376) ├──────────────────────────────┼────────────────────────────────┼──────────────────────────────────┼────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────┼──────────────┼─────────┼───────────────────────────────────────────────────────────────────────────────────────
          ┌─────────────┐        │           │  ┌───┐  │        ┌─────────┐      ┌─┴─┐├─────────┴┐┌───┐┌─────────────┐┌───┐     ┌───┐└─┬─┘├─────────┬┘└─┬─┘└───────────┬───┬───────────┘                              │                  ┌─────────┐   │                      ┌───┐       │                │       ┌────────────────────────┐┌───┐┌───────────────────────┐┌───┐  ┌──────────────────────┐   │  ┌───┐       │  ┌───┐  │  ┌───────────────────────┐                                       ┌───────────────┐    
q_4 -> 13 ┤ U2(-π/4,-π) ├────────┼───────────┼──┤ X ├──┼────────┤ U1(π/4) ├──────┤ X ├┤ U1(-π/4) ├┤ X ├┤ U2(0,1.641) ├┤ X ├──■──┤ X ├──■──┤ U1(π/4) ├───■──────────────┤ X ├─────────────────────────■────────────────┼───────────────■──┤ U1(π/4) ├───┼───────────────────■──┤ X ├──■────┼────────────────┼────■──┤ U1(-0.944196508933282) ├┤ X ├┤ U3(1.0866,-π,0.12636) ├┤ X ├──┤ U3(1.0866,0.81784,0) ├───┼──┤ X ├──■────┼──┤ X ├──┼──┤ U1(0.627327644292588) ├───■────────────────────────────■──────┤ U3(π,π/2,π/2) ├────
          └─────────────┘        │         ┌─┴─┐└─┬─┘  │        └┬───────┬┘      └───┘└──────────┘└─┬─┘└─────────────┘└─┬─┘  │  └─┬─┘     └─────────┘                  └─┬─┘            ┌─────────┐┌─┴─┐┌──────────┐┌─┴─┐┌─────────┐┌─┴─┐├─────────┴┐┌─┴─┐┌─────────────┐┌─┴─┐└─┬─┘┌─┴─┐┌─┴─┐┌──────────┐┌─┴─┐┌─┴─┐└┬──────────────────────┬┘└─┬─┘└───────────────────────┘└─┬─┘┌─┴──────────────────────┴┐  │  └─┬─┘  │  ┌─┴─┐└─┬─┘  │  ├───────────────────────┴┐┌─┴─┐┌──────────────────────┐┌─┴─┐┌───┴───────────────┴───┐
q_2 -> 14 ───────────────────────■─────────┤ X ├──┼────■─────────┤ U1(π) ├──────────────────────────┼───────────────────┼────┼────┼──────────────────────────────────────■──────────────┤ U2(0,π) ├┤ X ├┤ U1(-π/4) ├┤ X ├┤ U1(π/4) ├┤ X ├┤ U1(-π/4) ├┤ X ├┤ U2(0,-3π/4) ├┤ X ├──■──┤ X ├┤ X ├┤ U1(-π/4) ├┤ X ├┤ X ├─┤ U1(3.01523521327176) ├───■─────────────────────────────■──┤ U3(2.724,0.99285,1.685) ├──■────

### Approach #2: Monkey-patching

Python is allowing you to change definitions of classes and function in a runtime.
This is useful if you need to patch a small chunk of class with your implementation.
In our case we will want to swap chunks of code that can be run in parallel with our ray calls.

Let's create a dummy class that we will be monkey-patching later

In [8]:
class DummyClass:
    def sum_n_times(self, a: int, b: int, n_times: int):
        """Do something n times."""
        
        result = []
        for i in range(n_times):
            result.append(a + b)
        return result     

---
Let's write patch function

In [9]:
@run_qiskit_remote()
def sum_remote(a: int, b: int):
    return a + b

def sum_n_times_patch(self, a: int, b: int, n_times: int):
    return get([
        sum_remote(a, b)
        for _ in range(n_times)
    ])  

Patch it now

In [10]:
DummyClass.sum_n_times = sum_n_times_patch

Now we need to create instance of our patched class and run it

In [11]:
dummy_class = DummyClass()

with serverless:
    result = dummy_class.sum_n_times(1, 1, n_times=10)
    print(result)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


Function is leveraging parallelization internally now